In [ ]:
import wiggum as wg
import pandas as pd
import numpy as np
import timeit
import seaborn as sns
import warnings
from itertools import chain
warnings.filterwarnings('ignore')

In [ ]:
final=pd.DataFrame()

def test_scalability(labeled_df):

    times=[]
    for command in commands:
        for pearson_obj in objs:

            statement=command.format(pearson_obj)
            print(statement)
            t=timeit.repeat(statement, prereqs, repeat=10, number=100)
            times.append(t)
            print(t)
    timed = [ele for lis in times for ele in lis]
    return timed

def create_df(N, number_cluster, num_splitby, labeled_df, num_dep=None, num_indep=None, num_dep_indep=None):
    timings=test_scalability(labeled_df)
    function_names=['cct', 'get_subgroup_trends', 'get_trends (agg)', 'get_trends (sub)']
    result_df=pd.DataFrame(columns=['times', 'data', 'function'])
    # setting function column
    function=[]
    for ele in function_names:
        function.extend([ele]*20)
    # setting version column
    version=[['v1']*10, ['v2']*10]*len(function_names)
    version=list(chain.from_iterable(version))
    # setting trial column
    trial=list(range(10))*(len(function_names)*2)

    data_name='synthetic {}'.format(N)
    
    result_df['times']=timings
    result_df['function']=function
    result_df['data']=str(data_name)
    result_df['trial']=trial
    result_df['version']=version
    result_df['number_cluster']=number_cluster
    result_df['num_splitby']=num_splitby
    if num_dep_indep==None:
        result_df['independent']=num_indep
        result_df['dependent']=num_dep
    else:
        result_df['dep_indep_vars']=num_dep_indep
        
    return result_df


In [ ]:
import mlsim
from mlsim import sp_plot

In [ ]:
np.random.seed(20210627)


r_clusters =     [-.8, .5, .4, .7, -.6, .5, .4,-.9]  # magnitude correlation coefficient of clusters
cluster_spread = [ .3,-.2, .1,-.4, .2, -.1, .2,.4] # pearson correlation of means
p_sp_clusters =  [  1,  0,  0,  0,  1,   0,  0,  1] # portion of clusters with SP  #1 if r neg 0 if rpos
cluster_size =   [2,  3]#
domain_range = [0, 20, 0, 20] # of all data

In [ ]:
n_list = [1000, 5000, 10000]
number_cluster_list=[2, 4, 8, 16, 32]
num_dep_indep_list = [4, 8, 16]
num_splitby_list = [4, 8, 16]

In [ ]:
def generate_data(N, number_cluster, num_splitby, num_dep=None, num_indep=None, num_dep_indep=None):   
    k = [number_cluster]*40
    n_view = len(k)
    p_clusters = [[1/k_i]*k_i for k_i in k]
    many_sp_df = mlsim.geometric_indep_views_gmm_sp(n_view,r_clusters,cluster_size,cluster_spread,p_sp_clusters,
                domain_range,k,N,p_clusters)
    # print(many_sp_df.shape)
    labeled_df = wg.LabeledDataFrame(many_sp_df)

    n_view = int(len(labeled_df.df.columns) / 3)

    roles={'x1':['ignore']}
    if num_dep_indep==None:
        i=0
        while i<(n_view*2):
            if i<indep:
                roles.update({'x'+str(i+1):['independent']})
            elif i>indep and i<(indep+dep):
                roles.update({'x'+str(i+1):['dependent']})
            else:
                roles.update({'x'+str(i+1):['ignore']})
            i+=1
    else:
        # set dependent and independent for some xi, ignore for the rest
        dep_indep_list = np.random.choice(n_view*2, num_dep_indep)
        #print(dep_indep_list)
        roles = {'x'+str(i+1):['ignore'] if i not in dep_indep_list else ['independent','dependent'] 
                    for i in range(n_view*2)}
    # vars without 'x' in them are splitbys
    splitby_var_list = [cn for cn in labeled_df.df.columns if not('x' in cn)]
    # set splitby for some variable, ignore for the rest
    splitby_list = np.random.choice(splitby_var_list, num_splitby)
    roles.update( {c:['splitby'] if c in splitby_list else ['ignore'] for c in splitby_var_list})    

    count_list = []

    var_types = {'x'+str(i+1):'continuous' for i in range(n_view*2)}
    var_types.update( {c:'categorical' for c in splitby_var_list})
    weighting = {}

    labeled_df.set_counts(count_list)
    labeled_df.set_roles(roles)
    labeled_df.set_var_types(var_types)
    labeled_df.meta_df
    
    return labeled_df

In [ ]:
# symmetric cases
result=pd.DataFrame()
objs=['pearson', 'pearson_2']
prereqs='from __main__ import labeled_df, pearson, pearson_2, objs, commands'
commands=["{}.compute_correlation_table(labeled_df.df, 'agg_trend')",
          'labeled_df.get_subgroup_trends_1lev([{}])',
         "{}.get_trends(labeled_df.df, 'agg_trend')",
         "{}.get_trends(labeled_df.df, 'sub_trend')"]

for N in n_list:
    for number_cluster in number_cluster_list:
        for num_dep_indep in num_dep_indep_list:
            for num_splitby in num_splitby_list:
                labeled_df=generate_data(N, number_cluster, num_splitby, num_dep_indep=num_dep_indep)
                pearson = wg.All_Pearson()
                pearson.get_trend_vars(labeled_df)

                pearson_2 = wg.trends.All_Pearson_V2()
                pearson_2.get_trend_vars(labeled_df)
                result_df=create_df(N, number_cluster, num_splitby, labeled_df, num_dep_indep=num_dep_indep)
                result=pd.concat([result, result_df])
result.to_csv('data/sym_vars')

In [ ]:
n_list = [1000, 5000, 1000]
number_cluster_list=[2, 4, 8, 16, 32]
num_indep_list = [5, 10, 15, 20, 25]
num_dep_list = [5, 10, 15, 20, 25]
num_splitby_list = [4, 8, 16]

In [ ]:
# assymetric cases
result=pd.DataFrame()
indep=2
objs=['pearson', 'pearson_2']
prereqs='from __main__ import labeled_df, pearson, pearson_2, objs, commands'
commands=["{}.compute_correlation_table(labeled_df.df, 'agg_trend')",
          'labeled_df.get_subgroup_trends_1lev([{}])',
         "{}.get_trends(labeled_df.df, 'agg_trend')",
         "{}.get_trends(labeled_df.df, 'sub_trend')"]

while indep<=20:
    dep=2
    while dep<=20: 
        labeled_df=generate_data(N, number_cluster, dep, indep, num_splitby)
        pearson = wg.All_Pearson()
        pearson.get_trend_vars(labeled_df)

        pearson_2 = wg.trends.All_Pearson_V2()
        pearson_2.get_trend_vars(labeled_df)
        
        result_df=create_df(N, number_cluster, num_splitby, labeled_df, num_dep=dep, num_indep=indep, num_dep_indep=None)
        result=pd.concat([result, result_df])
        print('independents: ', indep)
        print('dependents: ', dep)
        dep+=2
    indep+=2
result.to_csv('data/aysm_vars')